In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../lite_data/india_July_21.csv", header=0)
data.head()

,Unnamed: 0,domain,id,id_str,party,screen_name,state
0,0,1.0,784325894559059968,784325894559059968,To Be Added,Sanjay_Meshack,To Be Added
1,1,1.0,741171197342187521,741171197342187520,BJP,BJP4AnN,Andaman and Nicobar Islands
2,2,1.0,1283244990274002949,1283244990274002944,INC,NSubramanian1,To Be Added
3,3,1.0,935363557050298368,935363557050298368,INC,SatishYepuri,Andhra Pradesh
4,4,1.0,112384119,112384119,BJP,praveenkubjp,Bihar


In [46]:
data.party.unique()

array(['To Be Added', 'BJP', 'INC', 'NCP', 'CPIM', 'AIMIM', 'MNS', 'AAP',
       'RPI', 'CPI', 'BSP', 'DMK', 'CPIML', 'GOV', 'LJP', 'HAM',
       'SHIV SENA', 'SP', 'AIADMK', 'YKD', nan, 'PRJA', 'NEIDP', 'NPP',
       'AITC', 'NDPP', 'NPF', 'BJD', 'LSP', 'RLD', 'YSRCP', 'TDP', 'VHP',
       'TRS', 'IND', 'JAPL', 'TJS', 'JDU', 'SAD', 'PEP', 'LIP', 'LDP',
       'RJD', 'PDF', 'LJD', 'PPI', 'RLSP', 'SWI', 'NUZP', 'RLP', 'SDPI',
       'HSP', 'SDF', 'SKM', 'AISMK', 'AMMK', 'DMDK', 'IJK', 'TMC', 'MMK',
       'MNM', 'PMK', 'PTP', 'VCK', 'IPFT', 'RSS', 'SPI', 'YDF', 'AD',
       'AIPF', 'ASP', 'BPP', 'BSCP', 'JSVP', 'LD', 'LOKSP', 'RJP', 'SBP',
       'SBSP', 'SHP', 'GJM', 'LEFT', 'JCC', 'JDS', 'TAMIL NADU', 'NOM',
       'AGP', 'AIUDF', 'INLD', 'IUML', 'JJP', 'JKNC', 'JKPDP', 'JMM',
       'GFP', 'JVP', 'AIP', 'JKNPP', 'JKPC', 'JKPM', 'AJSU', 'AKNJPSV',
       'PFI', 'AIFB', 'KC', 'AP', 'BJSP', 'RSAMAJP', 'ABVP', 'OTHER',
       'SAMAJWADI ', 'SWARAJ', 'HMK', 'KSP', 'SFI', 'SAMAJWADI',
    

In [3]:
data.sample(10)

,Unnamed: 0,domain,id,id_str,party,screen_name,state
22731,22731,NaN,995146329155252225,995146329155252224,To Be Added,OparmitaM,To Be Added
20643,20643,NaN,256889086,256889086,INC,tinucool1155,To Be Added
5503,5503,1.0,919883645539586049,919883645539586048,INC,SAbdulGaniRaja1,Tamil Nadu
21481,21481,NaN,1164449097866022912,1164449097866022912,AIADMK,Balamur46844419,To Be Added
3263,3263,1.0,2199037009,2199037009,BJP,bharatshigala,Gujarat
5492,5492,1.0,961547955923660800,961547955923660800,INC,ChennaiCenAIPC,Tamil Nadu
32186,32186,NaN,896022963635367936,896022963635367936,To Be Added,PushkarAhir32,To Be Added
11076,11076,1.0,1346963316,1346963316,BJP,8777paresh,Gujarat
22736,22736,NaN,1016014719264612352,1016014719264612352,INC,oBUTqEa5Drm9plL,To Be Added
32903,32903,NaN,327818415,327818415,INC,irajeshpurohit,To Be Added


In [44]:
!pip install --user config

In [5]:
import requests
import os
import json
import pandas as pd
import time
import glob

In [40]:
def create_url(user_id,next_token):
    # Replace with user ID below
    if next_token==None:
        return "https://api.twitter.com/2/users/{}/tweets?max_results=100&start_time=2020-05-25T17:00:00Z&end_time=2021-12-07T01:00:00Z".format(user_id)
    else:
        return "https://api.twitter.com/2/users/{}/tweets?max_results=100&start_time=2020-05-25T17:00:00Z&end_time=2021-12-07T01:00:00Z&pagination_token={}".format(user_id, next_token)
        

def get_params():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    return {"tweet.fields": "created_at,entities,in_reply_to_user_id,lang,public_metrics,source"}


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


In [41]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAMYx2gAAAAAAg3V4j7Msz7CEf%2BSUa4HjBoK3T5E%3DLir3OoEjxr5wrKKfYCaw6EeDcHhKLXIjkrrlr5UHZnFSmoOxJY"

In [42]:
idlist = ["1346963316"]#, "2199037009"]

#unames - ["8777paresh", "bharatshigala"]

In [43]:
for cid in idlist:
    url = create_url(user_id = cid, next_token=None)
    
    fname = "./" + cid+".csv"
    
    print(url)
    headers = create_headers(bearer_token)
    params = get_params()

    json_response = connect_to_endpoint(url, headers, params)
    time.sleep(1)
    if "data" in json_response and len(json_response['data'])>0: 
        with open(fname, "w") as tfile:
            for j in json_response['data']:
                json.dump(j, tfile)
                tfile.write("\n")

            while  "meta" in json_response and "next_token" in json_response['meta']:
                url = create_url(user_id = cid, next_token=json_response['meta']['next_token'])
                json_response = connect_to_endpoint(url, headers, params)
                time.sleep(1)
                if "data" in json_response and len(json_response['data']) >0:
                    for j in json_response['data']:
                        json.dump(j, tfile)
                        tfile.write("\n")


https://api.twitter.com/2/users/1346963316/tweets?max_results=100&start_time=2020-05-25T17:00:00Z&end_time=2021-12-07T01:00:00Z
200
200
200
200
200
200
200
200
200
